<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/EDA_with_multilingual_e5_large_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tqdm.auto import tqdm
import numpy as np

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

import torch
import torch.nn as nn
from torch.nn.functional import normalize

from datasets import load_dataset
from transformers import (
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
)
from transformers.modeling_outputs import ModelOutput

from trl import SFTTrainer

In [ ]:
dataset = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8, val_ratio=0.1, test_ratio=0.1,
    random_state=42,
    shuffle=True,
    trust_remote_code=True,
)
num_categories = len(set(dataset["train"]["category"]))
max_seq_length = 512

In [ ]:
category_names = ['movie-enter', 'it-life-hack', 'kaden-channel', 'topic-news', 'livedoor-homme', 'peachy', 'sports-watch', 'dokujo-tsushin', 'smax']

In [ ]:
model_id = "intfloat/multilingual-e5-large-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, max_seq_length=max_seq_length)
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=num_categories,
).to(0)

/home/masada/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at intfloat/multilingual-e5-large-instruct and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def accuracy(model, tokenizer, corpus, labels, batch_size=4):
    model.eval()
    num_correct_answers, num_answers = 0, 0
    for i in tqdm(range(0, len(corpus), batch_size)):
        texts = corpus[i:i+batch_size]
        encodings = tokenizer(texts, padding=True, return_tensors="pt")
        encodings = encodings.to(model.device)
        category = torch.tensor(labels[i:i+batch_size]).to(model.device)
        with torch.no_grad():
            outputs = model(**encodings)
        predicted = outputs.logits.argmax(-1)
        num_correct_answers += (predicted == category).sum()
        num_answers += len(texts)
    model.train()
    return (num_correct_answers / num_answers).item()

In [ ]:
def average_pool(last_hidden_states, attention_mask):
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [ ]:
def embed(model, tokenizer, corpus, batch_size=4):
    model.eval()
    pooled_hidden_states = []
    for i in tqdm(range(0, len(corpus), batch_size)):
        texts = corpus[i:i+batch_size]
        encodings = tokenizer(texts, padding=True, return_tensors="pt")
        encodings = encodings.to(model.device)
        with torch.no_grad():
            outputs = model.roberta(**encodings)
        pooled_hidden_state = average_pool(
            outputs.last_hidden_state,
            encodings['attention_mask'],
        )
        pooled_hidden_states.append(pooled_hidden_state.cpu())
    model.train()
    return torch.cat(pooled_hidden_states)

In [ ]:
embeddings = {}
for key in dataset:
    embeddings[key] = embed(model, tokenizer, dataset[key]["title"])
    embeddings[key] = normalize(embeddings[key])

  0%|          | 0/1474 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

In [ ]:
n_clusters = 30
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=123)
kmeans.fit(embeddings["train"])
centers = kmeans.cluster_centers_

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
size_dict = dict(zip(unique, counts))
print(sorted([item[1] for item in size_dict.items()]))

[68, 79, 84, 101, 103, 132, 135, 136, 137, 157, 160, 162, 163, 170, 175, 176, 200, 216, 217, 227, 227, 240, 254, 273, 285, 290, 296, 306, 339, 386]


In [ ]:
label_pos_tags = ["NOUN", "VERB", "PROPN"]

nlp = spacy.load("ja_core_news_sm")
corpus = {}
for key in dataset:
    corpus[key] = []
    for text in tqdm(dataset[key]["title"]):
        corpus[key].append(" ".join(
            [token.lemma_
             for token in nlp(text) if token.pos_ in label_pos_tags
            ]
        ))

  0%|          | 0/5894 [00:00<?, ?it/s]

  0%|          | 0/737 [00:00<?, ?it/s]

  0%|          | 0/736 [00:00<?, ?it/s]

In [ ]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.1, lowercase=False)
vectorizer.fit(corpus["train"])
vocab = np.array(vectorizer.get_feature_names_out())

In [ ]:
vocab

array(['1人', '1日', '2人', 'AQUOS', 'Android', 'CM', 'CPU', 'DVD', 'Excel',
       'Facebook', 'Fi', 'GHz', 'GX', 'Google', 'HD', 'HT', 'HTC', 'ICS',
       'IS', 'IT', 'In', 'KDDI', 'LAN', 'Mac', 'Mobile', 'NHK', 'NTT',
       'No', 'OK', 'OS', 'Office', 'PC', 'Play', 'REPORT', 'SC', 'SH',
       'SII', 'SNS', 'SO', 'SPORTS', 'SX', 'TV', 'Twitter', 'UP', 'USB',
       'Ubuntu', 'VS', 'Vol', 'Wi', 'Windows', 'Word', 'W杯', 'akb',
       'android', 'arrows', 'au', 'by', 'cafe', 'eluga', 'for', 'galaxy',
       'hulu', 'iOS', 'iPhone', 'iii', 'ipad', 'isw', 'jojo', 'lte',
       'medias', 'mm', 'note', 'nottv', 'optimus', 'pantone', 'peachy',
       'phone', 'presented', 'regza', 'salon', 'siii', 'sim', 'tab',
       'touch', 'ultrabook', 'watch', 'web', 'wimax', 'xi', 'xperia',
       'あらわれる', 'あり', 'ある', 'いう', 'いく', 'いける', 'いる', 'おく', 'お気に入り', 'かける',
       'くさる', 'くす', 'くる', 'くれる', 'こと', 'さん', 'しまう', 'すぎる', 'する', 'たち',
       'ため', 'ちゃん', 'つく', 'つける', 'できる', 'とき', 'とも', 'どっち', 'なし', 'なでし

In [ ]:
vocab_embeddings = embed(model, tokenizer, list(vocab))

  0%|          | 0/213 [00:00<?, ?it/s]

In [ ]:
topic_words = []
similarities = cosine_similarity(vocab_embeddings, centers)
for i in range(similarities.shape[-1]):
    indices = np.argsort(- similarities[:,i])
    topic_words.append(f"{i:d} " + " ".join(list(vocab[indices[:20]])))
print("\n".join(topic_words))

0 インテル パソコン シャープ デジ パナソニック 搭載 デュアルコア ソニー モバステ クアッドコア コレ はじめる 機種 トップ サイフ キーボード ニコニコ ホーム 楽天 超える
1 コレ すぎる しまう 騒動 負ける なる ダメ たち 感ずる ワケ どっち アナタ 込む さん 物議 騒然 ツイッター する なでしこ 超える
2 アナタ なでしこ 真央 エリカ コレ 優子 オンナ モテる 超える たち 愛す ヒロイン さん くれる みる アナ ドキドキ しまう 真司 イケメン
3 モテる しまう アナタ 愛す 超える なれる オンナ 込む コレ 彼氏 週間 お気に入り トップ なでしこ くす イケショップ 感ずる ベスト イケメン 今週
4 イケショップ モバステ ケータイ ドコモ スマホ シャープ 楽天 スマートフォン バッテリー コレ タブレット 携帯 ソニー ソフトバンク 端末 充電 アップル 搭載 モバイル パナソニック
5 コレ なでしこ くれる イケショップ アナタ バレンタイン らくらく ホーム お気に入り 愛す しまう にゅう ベルセルク 癒す よめ エリカ 超える 過ごす ゲット トップ
6 コレ アナタ しまう 超える なる たち くれる なでしこ 込む おく する よる なれる どっち さん みる ウラ くす 虎の巻 感ずる
7 コレ 選手 負ける なでしこ ワケ 圭佑 観る どっち アナタ 超える ツイッター なる しまう 狙う たち プレーヤー みる W杯 見る よる
8 虎の巻 裏技 はじめる まとめる トップ 使い方 コレ ワザ しまう フラッシュバック まとめ 入れる サイフ 込む 付ける 隠す する ツイッター 使う ニコニコ
9 虎の巻 ハリウッド 上映 アベンジャーズ ムービー アナタ 映画 コレ 超える 観る 必見 ディズニー フジテレビ フラッシュバック 登場 復活 ワンセグ 主演 新作 なでしこ
10 ニコニコ ツイッター ツイート コレ ネット はじめる トップ しまう サイト する ホーム アナタ 楽天 ブログ インターネット サイフ どっち みる 超える モバステ
11 コレ アナタ なでしこ しまう 負ける 泣く 真央 ワケ たち 超える なる すぎる エリカ 虎の巻 さん オンナ 真司 込む フジテレビ 

In [ ]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.1, lowercase=False)
vectorizer.fit(corpus["train"])
vocab = np.array(vectorizer.get_feature_names_out())
X_train = vectorizer.transform(corpus["train"]).toarray()

In [ ]:
vocab_embeddings = np.dot((X_train / X_train.sum(0)).T, embeddings["train"])

In [ ]:
topic_words = []
similarities = cosine_similarity(vocab_embeddings, centers)
for i in range(similarities.shape[-1]):
    indices = np.argsort(- similarities[:,i])
    topic_words.append(f"{i:d} " + " ".join(list(vocab[indices[:20]])))
print("\n".join(topic_words))

0 PC デジ パソコン 登場 新型 ノート 製品 シリーズ 使える 機能 HD ultrabook 液晶 Windows 使う ディスプレイ データ 発売 対応 販売
1 批判 殺到 非難 物議 続出 次ぐ 発言 ネット 賛否 騒動 怒り 掲示 激怒 コメント 波紋 問題 受ける 疑問 炎上 ユーザー
2 さん 披露 美女 ある いく いる こと インタビュー くる 見る ちゃん 美人 出演 女優 注目 すぎる 少女 人気 する デビュー
3 週間 ランキング ライフスタイル 登録 お気に入り 記事 ビューティー 恋愛 行動 方法 ビューティ 彼氏 効く 作る なれる みんな つける 女子 なる する
4 ケース イケショップ レア 専用 カバー スマホ バッテリー チェック 使える for iPhone 充電 グッズ アイテム 容量 機能 使う 紹介 登場 売れ筋
5 気分 楽しむ プレゼント 入れる 行く 本格 人気 癒す ため 大人 なる くれる 贈る 期間 誕生 今年 特集 注目 作る 見る
6 ある いう する こと いる なる つく 出る 思う いく やる もの 見る 取る さん くる 時代 くれる 変わる 出す
7 watch SPORTS 選手 試合 監督 プロ 代表 星野 チーム 野球 巨人 語る なでしこ 本田 サッカー 開幕 岡田 ノム スポーツ アナ
8 虎の巻 得る ワザ 知る Word ファイル Excel テクニック 便利 入力 管理 表示 役立つ 操作 活用 裏技 使う 方法 使い方 Facebook
9 映画 作品 主演 上映 公開 国際 ストーリー プレミア 描く 映像 アカデミー 解禁 まとめ 感動 受賞 決定 ハリウッド 編集 超える 予告
10 情報 話題 利用 サービス 使う わかる みる サイト なる よる できる つく メディア web あり ユーザー ニュース 公式 使える する
11 さん 発言 対する 騒然 暴露 出演 ファン コメント いく 出る いる 騒動 告白 明かす 続出 受ける akb 関係 次ぐ 報道
12 男性 いる もの 事情 する 本当 なる しまう 思う ある こと アリ 女性 とき 聞く 結婚 いう 自分 求める たち
13 開始 実施 サービス 通信 キャンペーン 端末 利用 向け モバ

In [ ]:
class MyNetForClassification(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        self.pretrained = pretrained
        self.config = self.pretrained.config

    def forward(
        self, input_ids, category=None,
        attention_mask=None,
        output_attentions=None, output_hidden_states=None,
        return_dict=None, inputs_embeds=None, labels=None,
    ):
        outputs = self.pretrained(
            input_ids,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(outputs.logits, category)
        return ModelOutput(
            loss=loss,
            logits=outputs.logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )
my_model = MyNetForClassification(model)

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    output_dir="outputs",
    label_names=["category"],
    max_steps=300,
    eval_steps=100,
    logging_steps=100,
    save_steps=100,
    learning_rate=5e-5,
    optim_target_modules=["query", "key", "value", "dense"],
    evaluation_strategy="steps",
    logging_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
)

In [ ]:
trainer = SFTTrainer(
    model=my_model,
    args=training_args,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    dataset_text_field="title",
)
trainer.train_dataset = trainer.train_dataset.add_column(
    "category", dataset["train"]["category"],
)
trainer.eval_dataset = trainer.eval_dataset.add_column(
    "category", dataset["validation"]["category"],
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
100,0.866900,0.416719
200,0.432000,0.288546
300,0.211800,0.302716


TrainOutput(global_step=300, training_loss=0.5035534222920736, metrics={'train_runtime': 133.7274, 'train_samples_per_second': 71.788, 'train_steps_per_second': 2.243, 'total_flos': 0.0, 'train_loss': 0.5035534222920736, 'epoch': 1.628222523744912})

In [ ]:
accuracy(model, tokenizer, dataset["validation"]["title"], dataset["validation"]["category"])

  0%|          | 0/185 [00:00<?, ?it/s]

0.9063771963119507

In [ ]:
embeddings = {}
for key in dataset:
    embeddings[key] = embed(model, tokenizer, dataset[key]["title"])
    embeddings[key] = normalize(embeddings[key])

  0%|          | 0/1474 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

In [ ]:
vocab_embeddings = np.dot((X_train / X_train.sum(0)).T, embeddings["train"])

In [ ]:
n_clusters = 30
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=123)
kmeans.fit(embeddings["train"])
centers = kmeans.cluster_centers_

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
size_dict = dict(zip(unique, counts))
print(sorted([item[1] for item in size_dict.items()]))

[26, 68, 92, 92, 106, 109, 125, 127, 133, 134, 135, 140, 159, 163, 173, 174, 178, 187, 187, 192, 197, 243, 270, 275, 280, 305, 330, 350, 387, 557]


In [ ]:
topic_words = []
similarities = cosine_similarity(vocab_embeddings, centers)
for i in range(similarities.shape[-1]):
    indices = np.argsort(- similarities[:,i])
    topic_words.append(f"{i:d} " + " ".join(list(vocab[indices[:20]])))
print("\n".join(topic_words))
#with open("topic_words.txt", "w") as f:
#    f.write("\n".join(topic_words))

0 デジ Ubuntu Mac ソフト ダウンロード PC デバイス 便利 液晶 データ クラウド 使える 管理 容量 IT ultrabook イケショップ レア トラブル 募集
1 明かす 美女 絶賛 少女 披露 日本 さん マン ぶり ファン 連発 告白 笑う 語る 出演 天才 優子 大島 アナ ニュー
2 ガール コレクション ガールズ No 贈る 招待 インタビュー 豪華 試写 スポット 誕生 終了 会える 美人 よう かける 気分 ホテル セクシー ファッション
3 売れ筋 話題 ビデオ アップル salon チェック hulu 連載 連動 インターネット 売れる 家電 発生 パナソニック 今度 原因 プレーヤー 電気 ソニー 節電
4 独女 オトナ コミック presented ゆるい cafe オンナ by 悩み ひとり はじめる よめ あらわれる くさる たち 風呂 しまう 自転車 男性 女子
5 ねこ android アプリ 画面 Android ICS バージョン SC 提供 赤外 NTT デュアルコア ワンセグ xi SO HT nottv HTC ソフトウェア ドコモ
6 説教 辛口 ビジネスマン 部屋 年収 人事 転職 ウラ 活動 スーツ 図鑑 オトコ 研究 ビジネス 会社 Vol プレイヤー 生活 特集 採用
7 SPORTS watch 星野 斎藤 巨人 ダルビッシュ 試合 本田 岡田 ノム 開幕 長友 圭佑 W杯 引退 移籍 野球 落合 なでしこ チーム
8 NTT wimax 提供 isw ドコモ xi phone SH 更新 ソフトウェア 具合 arrows イー HT 開始 AQUOS IS 赤外 ICS lte
9 ベルセルク 人類 ポスター 予告 解禁 来日 スター ハリウッド 対決 アベンジャーズ ナイト ヒロイン 公開 ダーク 劇場 恐怖 上陸 主題 映像 奇跡
10 続出 次ぐ 物議 騒然 賛否 非難 発言 両論 殺到 暴露 対する 掲示 反応 騒動 中島 批判 怒り 波紋 橋下 卒業
11 ultrabook PC マザー ノート ソフト レビューアー Mac 募集 IT フラッシュバック 新型 ニコニコ ファイル ドライブ クラウド 会議 Ubuntu 得る 虎の巻 ワザ
12 物議 掲示 非難 殺到 次ぐ 橋下

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.to("cpu").eval();

In [ ]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

token_reference = TokenReferenceBase(reference_token_idx=tokenizer.pad_token_id)

In [ ]:
text = dataset["train"]["title"][0]
encodings = tokenizer(text, padding=True, return_tensors="pt")
encodings = encodings.to(model.device)
outputs = model.roberta(encodings.input_ids, encodings.attention_mask)
pooled_hidden_state = average_pool(
    outputs.last_hidden_state,
    encodings['attention_mask'],
)

In [ ]:
pooled_hidden_state

tensor([[ 0.7808, -0.8165, -0.7846,  ..., -0.6763, -0.1170,  0.0062]])

In [ ]:
cluster_centers = torch.tensor(kmeans.cluster_centers_, device=model.device)

In [ ]:
cos_sim = nn.CosineSimilarity(dim=-1)
cos_sim(cluster_centers, pooled_hidden_state)

tensor([0.5141, 0.2207, 0.2385, 0.0257, 0.0848, 0.4722, 0.2661, 0.1372, 0.5814,
        0.2020, 0.0792, 0.6513, 0.1441, 0.5768, 0.1106, 0.3512, 0.1776, 0.1458,
        0.6682, 0.5661, 0.5560, 0.1720, 0.0815, 0.1662, 0.3690, 0.0683, 0.5358,
        0.2173, 0.1252, 0.6922], dtype=torch.float64)

In [ ]:
kmeans.labels_

array([29, 16,  6, ...,  2,  3, 11], dtype=int32)

In [ ]:
def predict(input_ids, attention_mask):
    outputs = model.roberta(input_ids, attention_mask)
    pooled_hidden_state = average_pool(
        outputs.last_hidden_state,
        attention_mask,
    )
    return cos_sim(
        cluster_centers.unsqueeze(0),
        pooled_hidden_state.unsqueeze(1)
    )

In [ ]:
text = dataset["train"]["title"][0]
encodings = tokenizer(text, padding=True, return_tensors="pt")
encodings.to(model.device)
predict(
    encodings.input_ids,
    encodings.attention_mask,
)

tensor([[0.5141, 0.2207, 0.2385, 0.0257, 0.0848, 0.4722, 0.2661, 0.1372, 0.5814,
         0.2020, 0.0792, 0.6513, 0.1441, 0.5768, 0.1106, 0.3512, 0.1776, 0.1458,
         0.6682, 0.5661, 0.5560, 0.1720, 0.0815, 0.1662, 0.3690, 0.0683, 0.5358,
         0.2173, 0.1252, 0.6922]], dtype=torch.float64)

In [ ]:
text

'トレンドマイクロのサービス「SafeSync」を無料体験して「横浜スタジアム 半日貸切権」や「デジカメ」を当てよう！'

In [ ]:
def cluster_similarity_forward_func(input_ids, attention_mask, cluster_id):
    similarities = predict(input_ids, attention_mask)
    return similarities[:,cluster_id]

In [ ]:
text = dataset["train"]["title"][0]
encodings = tokenizer(text, padding=True, return_tensors="pt")
encodings.to(model.device)
cluster_similarity_forward_func(
    encodings.input_ids,
    encodings.attention_mask,
    29,
)

tensor([0.6922], dtype=torch.float64)

In [ ]:
lig = LayerIntegratedGradients(
    cluster_similarity_forward_func,
    model.roberta.embeddings.word_embeddings,
)

In [ ]:
vis_data_records_ig = []

In [ ]:
def add_attributions_to_visualizer(attributions, text, pred_prob, pred_class, true_class,
                                   attr_class, convergence_scores, vis_data_records):
    attributions = attributions.cpu()
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()
    vis_data_records.append(
        visualization.VisualizationDataRecord(
            attributions,
            pred_prob,
            pred_class,
            true_class,
            attr_class,
            attributions.sum(),
            text,
            convergence_scores,
        )
    )

In [ ]:
def interpret_text(text, attr_class=None, n_steps=50):
    encodings = tokenizer(text, padding=True, return_tensors="pt")
    encodings = encodings.to(model.device)
    input_ids = encodings.input_ids
    attention_mask = encodings.attention_mask
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    reference_input_ids = token_reference.generate_reference(
        len(tokens),
        device=model.device,
    ).unsqueeze(0)

    similarities = predict(
        input_ids,
        attention_mask,
    )
    prediction = similarities.argmax().item()
    if attr_class is None:
        attr_class = prediction
    print(
        f"prediction={prediction} "
        f"cos_sim={similarities.max().item():.3f} ",
        end=""
    )

    attributions_ig, delta = lig.attribute(
        input_ids,
        reference_input_ids,
        additional_forward_args=(attention_mask, attr_class),
        n_steps=n_steps,
        return_convergence_delta=True,
    )
    print(f"convergence delta={delta.item():.3e} when n_steps={n_steps}")

    add_attributions_to_visualizer(
        attributions_ig,
        tokens,
        similarities.max().item(),
        str(prediction),
        str(prediction),
        str(attr_class),
        delta,
        vis_data_records_ig,
    )
    return prediction


In [ ]:
vis_data_records_ig = []
for n_steps in [50, 100, 200, 300]:
    interpret_text(dataset["train"]["title"][0], n_steps=n_steps)

prediction=29 cos_sim=0.692 convergence delta=-5.628e-01 when n_steps=50

prediction=29 cos_sim=0.692 convergence delta=-5.898e-01 when n_steps=100

prediction=29 cos_sim=0.692 convergence delta=-5.451e-01 when n_steps=200

prediction=29 cos_sim=0.692 convergence delta=-6.311e-01 when n_steps=300



In [ ]:
visualization.visualize_text(vis_data_records_ig);

In [ ]:
for i in tqdm(range(60, 70)):
    example = dataset["validation"][i]
    print(category_names[example["category"]], end=" ")
    vis_data_records_ig = []
    prediction = interpret_text(example["title"], n_steps=50)
    print("\t" + topic_words[prediction])
    visualization.visualize_text(vis_data_records_ig);

  0%|          | 0/10 [00:00<?, ?it/s]

sports-watch prediction=21 cos_sim=0.947 convergence delta=-5.635e-02 when n_steps=50
	21 選手 チーム 代表 真司 香川 投手 なでしこ 落合 移籍 サッカー 田中 真央 試合 松井 野球 アナ 引退 本田 ロンドン 石井


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
21,21 (0.95),21,3.07,#s ▁「 説 教 ですよ 」 ▁ な で し こ 丸 山 、 北京 五 輪 時 における 澤 の 助 言 とは #/s


peachy prediction=17 cos_sim=0.912 convergence delta=-2.370e-02 when n_steps=50
	17 プレゼント 終了 ビューティ セット バレンタイン ランキング 美肌 週間 メイク スイーツ ケーキ 登録 コスメ ドキドキ 気分 ビューティー ライフスタイル ガールズ お気に入り 招待


peachy prediction=14 cos_sim=0.834 convergence delta=-1.142e-01 when n_steps=50
	14 コスメ メイク トレンド 美容 美肌 UP ビューティー peachy バレンタイン ダイエット 登録 クリスマス ランキング スイーツ ケア ライフスタイル 週間 気分 お気に入り 愛す


...

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
12,12 (0.92),12,3.14,#s ▁ 教授 の ′′ 酷 すぎる ′′ 発言 に 批判 殺 到 #/s


it-life-hack prediction=13 cos_sim=0.746 convergence delta=-3.337e-02 when n_steps=50
	13 カバー サイト 調査 ドライブ ショー 登場 支援 アナタ レビューアー 募集 公式 オリジナル 活用 新型 便利 最新 必須 ノート 社長 情報


movie-enter prediction=9 cos_sim=0.728 convergence delta=-1.197e-01 when n_steps=50
	9 ベルセルク 人類 ポスター 予告 解禁 来日 スター ハリウッド 対決 アベンジャーズ ナイト ヒロイン 公開 ダーク 劇場 恐怖 上陸 主題 映像 奇跡


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
9,9 (0.73),9,2.11,#s ▁ 一緒に 逃避 行 した くなる ? ▁ 脱 獄 映画 に 名 作 あり #/s


peachy prediction=25 cos_sim=0.984 convergence delta=-5.016e-02 when n_steps=50
	25 週間 ランキング ライフスタイル ビューティー ビューティ お気に入り 登録 美肌 メイク コスメ プレゼント ケア UP 終了 スイーツ トレンド バレンタイン peachy クリスマス ガールズ
